# Chemical Space Network Calculations

**V.F. Scalfani, V.D. Patel, and A.M. Fernandez** \
v. October 18, 2022

*w/ glucocorticoid_receptor_2034_2.csv dataset*

## 1. Import RDKit, Networkx, and other libraries

In [1]:
# RDKit stuff
from rdkit import Chem
from rdkit import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdDepictor
rdDepictor.SetPreferCoordGen(True)
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import rdFMCS
from rdkit import DataStructs
from rdkit.Chem import rdmolops

# numpy
import numpy as np

# pandas
import pandas as pd

# networkx
import networkx as nx

# matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# Print versions of libraries used
print('RDKit version: ',rdBase.rdkitVersion)
print('Numpy version:', np.__version__)
print('Pandas version:', pd.__version__)
print('Networkx version',nx.__version__)
print('MatplotLib version:', mpl.__version__)

RDKit version:  2022.03.5
Numpy version: 1.23.3
Pandas version: 1.4.4
Networkx version 2.8.6
MatplotLib version: 3.5.3


## 2. Load ChEMBL Dataset

In [3]:
# pd.options.display.max_rows = 30
df = pd.read_csv('glucocorticoid_receptor_2034_2.csv', sep =';')
df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL318409,NaN,0,387.91,1,7.10,1,CC1=CC(C)(C)Nc2ccc3c(c21)C(c1ccccc1)Oc1ccc(Cl)...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1134574,1,Scientific Literature,J. Med. Chem.,2001,None,NaN
1,CHEMBL427265,NaN,0,386.46,0,2.91,22,N#Cc1cccc(Cc2c(N3CCC(c4ccccc4)CC3)[nH]c(=O)[nH...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1146208,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2007,None,NaN
2,CHEMBL231500,NaN,0,418.50,0,3.00,31,CC(=O)Nc1ccc(C2CCN(c3[nH]c(=O)[nH]c(=O)c3Cc3cc...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1146208,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2007,None,NaN
3,CHEMBL230461,NaN,0,405.50,0,2.24,9,Cn1c(=O)[nH]c(N2CCC(O)(Cc3ccccc3)CC2)c(Cc2cccc...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1146208,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2007,None,NaN
4,CHEMBL410037,NaN,0,510.61,1,4.27,73,O=S(=O)(c1ccc(F)cc1)N1CCC2=Cc3c(cnn3-c3ccc(F)c...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1142367,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2008,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,CHEMBL2426660,NaN,0,453.57,1,5.67,5b,CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2c(cnn2-c2...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL2424628,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2013,None,NaN
455,CHEMBL2426653,NaN,0,469.57,1,5.38,5l,CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2c(cnn2-c2...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL2424628,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2013,None,NaN
456,CHEMBL2426663,NaN,0,471.56,1,5.81,(3R)-5c,CC(C)(C(=O)Nc1nncs1)[C@H](c1ccccc1)c1ccc2c(cnn...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL2424628,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2013,None,NaN
457,CHEMBL3735496,NaN,0,528.61,1,4.62,27,Cc1ccc(S(=O)(=O)N2CCC3=Cc4c(cnn4-c4ccc(F)cc4)C...,Ki,'=',...,Glucocorticoid receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL3734698,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2015,None,NaN


# 3. Data Preparation and Checks

Note that we are using the chemical structures as-is (i.e., using ChEMBL's standardization workflow). See supporting manuscript for more details. 

In [4]:
# Create a new dataframe with only Chembl ID, Smiles, and Standard Value (Ki, for this example)
# We are ignoring the Standard relation here for Ki values (i.e., >, <, etc. are treated as =)

df1 = df[['Molecule ChEMBL ID','Smiles','Standard Value']].copy()

# drop any rows with NaN (missing Standard Values)
df1.dropna(inplace=True)
df1

,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL318409,CC1=CC(C)(C)Nc2ccc3c(c21)C(c1ccccc1)Oc1ccc(Cl)...,9.30
1,CHEMBL427265,N#Cc1cccc(Cc2c(N3CCC(c4ccccc4)CC3)[nH]c(=O)[nH...,37.00
2,CHEMBL231500,CC(=O)Nc1ccc(C2CCN(c3[nH]c(=O)[nH]c(=O)c3Cc3cc...,46.00
3,CHEMBL230461,Cn1c(=O)[nH]c(N2CCC(O)(Cc3ccccc3)CC2)c(Cc2cccc...,360.00
4,CHEMBL410037,O=S(=O)(c1ccc(F)cc1)N1CCC2=Cc3c(cnn3-c3ccc(F)c...,1.60
...,...,...,...
454,CHEMBL2426660,CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2c(cnn2-c2...,5.00
455,CHEMBL2426653,CC(C)(C(=O)Nc1nncs1)C(c1ccccc1)c1ccc2c(cnn2-c2...,7.20
456,CHEMBL2426663,CC(C)(C(=O)Nc1nncs1)[C@H](c1ccccc1)c1ccc2c(cnn...,6.70
457,CHEMBL3735496,Cc1ccc(S(=O)(=O)N2CCC3=Cc4c(cnn4-c4ccc(F)cc4)C...,0.22


In [5]:
# Check for presence of disconnected SMILES notation via string matching 
df2 = df1[~df1['Smiles'].str.contains("\.")]
len(df2) # same as df1

454

In [6]:
# We should double check for disconnected fragments in the event that
# the dot disconnect bond is used with ring-closures
# see: http://www.dalkescientific.com/writings/diary/archive/2004/12/12/library_generation_with_smiles.html

smis = df1['Smiles'].tolist()
num_frags = []
for smi in smis:
    mol = Chem.MolFromSmiles(smi)
    num_frags.append(len(Chem.GetMolFrags(mol))) # returns number of fragments

In [7]:
# now check that all molecules have only one fragment
all(frag == 1 for frag in num_frags)

True

In [8]:
# Group by ChEMBLID/Smiles rows, and then take the mean of the standard Ki value to account for duplicates.
# See: Zhang, B. et al. J Comput Aided Mol Des 2015, 29 (10), 937–950. 
# https://doi.org/10.1007/s10822-015-9872-1.
df3 = df2.groupby(['Molecule ChEMBL ID', 'Smiles'])['Standard Value'].mean().reset_index()

# rename the Standard Value Column to Ki
df3.rename(columns={'Standard Value': 'Ki'}, inplace=True)
df3

,Molecule ChEMBL ID,Smiles,Ki
0,CHEMBL103,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,30.500000
1,CHEMBL1089811,COc1ccc2c(c1)-c1ccc3c(c1[C@H](c1ccccc1)O2)C(C)...,18.000000
2,CHEMBL1089812,C#COc1ccc2c(c1OC)-c1ccc3c(c1[C@H](CC=C)O2)C(C)...,4.200000
3,CHEMBL1090477,CC#C[C@@]1(O)CC[C@@]2(Cc3ccccc3)c3ccc(C(=O)NCc...,2.000000
4,CHEMBL1090801,C=CC[C@@H]1Oc2ccc(OC#N)c(OC)c2-c2ccc3c(c21)C(C...,51.000000
...,...,...,...
399,CHEMBL8639,C=CCC1Oc2ccccc2-c2ccc3c(c21)C(C)=CC(C)(C)N3,889.000000
400,CHEMBL95644,COc1cccc2c1-c1ccc3c(c1C(c1cccc(C(F)(F)F)c1)O2)...,11.000000
401,CHEMBL97206,COc1cccc2c1-c1ccc3c(c1C(c1cccc(OCSC)c1)O2)C(C)...,4.000000
402,CHEMBL99046,COc1cccc2c1-c1ccc3c(c1C(c1ccccc1)O2)C(C)=CC(C)...,82.266667


In [9]:
# Double check that all ChEMBL IDs are now unique
chembl_ids = df3['Molecule ChEMBL ID'].tolist()

# create a set, if the set length matches the list, there are no duplicates
set_chembl_ids = set(chembl_ids)
print(len(set_chembl_ids) == len(chembl_ids))

True


In [10]:
# Double check that all SMILES are unique (different) compounds
# To be on the safe side, we can parse the SMILES as RDKit mol objects
# then write out canonical smiles and check

smis = df3['Smiles'].tolist()
rdkit_can_smiles = []
for smi in smis:
    mol = Chem.MolFromSmiles(smi)       
    rdkit_can_smiles.append(Chem.MolToSmiles(mol, canonical=True)) # default is true
    
set_rdkit_can_smiles = set(rdkit_can_smiles)
len(set_rdkit_can_smiles) == len(rdkit_can_smiles)

True

In [11]:
# We'll use the original SMILES as unique dictionary keys, so we should verify that the
# ChEMBL SMILES are unique strings too.
set_smis = set(smis)
len(set_smis) == len(smis)

True

## 4. Compile Node data

In [12]:
# We will use pKi for coloring nodes.
from math import log10

# Ki values are in nM units
# 1. convert to M
df3.loc[:,"Ki_M"] = (df3.loc[:,"Ki"] * (10**-9))

# 2. then compute -log10[Ki]
def minuslog(x):
    return -log10(x)

df3.loc[:,"pKi"] = (df3.loc[:,"Ki_M"].apply(minuslog))

In [13]:
# drop Ki and Ki_M columns (no longer needed)
df3.drop(["Ki","Ki_M"],axis=1,inplace=True)
df3

,Molecule ChEMBL ID,Smiles,pKi
0,CHEMBL103,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,7.515700
1,CHEMBL1089811,COc1ccc2c(c1)-c1ccc3c(c1[C@H](c1ccccc1)O2)C(C)...,7.744727
2,CHEMBL1089812,C#COc1ccc2c(c1OC)-c1ccc3c(c1[C@H](CC=C)O2)C(C)...,8.376751
3,CHEMBL1090477,CC#C[C@@]1(O)CC[C@@]2(Cc3ccccc3)c3ccc(C(=O)NCc...,8.698970
4,CHEMBL1090801,C=CC[C@@H]1Oc2ccc(OC#N)c(OC)c2-c2ccc3c(c21)C(C...,7.292430
...,...,...,...
399,CHEMBL8639,C=CCC1Oc2ccccc2-c2ccc3c(c21)C(C)=CC(C)(C)N3,6.051098
400,CHEMBL95644,COc1cccc2c1-c1ccc3c(c1C(c1cccc(C(F)(F)F)c1)O2)...,7.958607
401,CHEMBL97206,COc1cccc2c1-c1ccc3c(c1C(c1cccc(OCSC)c1)O2)C(C)...,8.397940
402,CHEMBL99046,COc1cccc2c1-c1ccc3c(c1C(c1ccccc1)O2)C(C)=CC(C)...,7.084776


In [14]:
# get the max/min Ki values, which will be used for a color bar later
print(df3.pKi.max())
print(df3.pKi.min())

10.096910013008056
4.0


In [15]:
# set the dataframe index as Smiles (we already verified they are all unique from eachother)
df4 = df3.set_index('Smiles')

In [16]:
pd.set_option("expand_frame_repr", False)
print(df4.head(10)) # view first 10

                                                   Molecule ChEMBL ID       pKi
Smiles                                                                         
CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C...          CHEMBL103  7.515700
COc1ccc2c(c1)-c1ccc3c(c1[C@H](c1ccccc1)O2)C(C)=...      CHEMBL1089811  7.744727
C#COc1ccc2c(c1OC)-c1ccc3c(c1[C@H](CC=C)O2)C(C)=...      CHEMBL1089812  8.376751
CC#C[C@@]1(O)CC[C@@]2(Cc3ccccc3)c3ccc(C(=O)NCc4...      CHEMBL1090477  8.698970
C=CC[C@@H]1Oc2ccc(OC#N)c(OC)c2-c2ccc3c(c21)C(C)...      CHEMBL1090801  7.292430
C#Cc1cccc2c1-c1ccc3c(c1[C@H](CC=C)O2)C(C)=CC(C)...      CHEMBL1090805  8.161151
C[C@]12C[C@H](O)[C@H]3C(CCC4=CC(=O)CC[C@@]43C)[...      CHEMBL1092578  7.522879
C=C1C(C)CCC(C)(C)[C@@H]1Cc1cc(OC)c(Br)cc1OC(C)=O        CHEMBL1093140  5.386264
C=C1CCC(Br)C(C)(C)[C@H]1Cc1cc(OC)c(Br)cc1O              CHEMBL1093625  5.608183
C=C1CCC(Br)C(C)(C)[C@@H]1Cc1cc(OC)c(Br)cc1O             CHEMBL1093779  5.493495


In [17]:
# save to a dictionary
node_data = df4.to_dict('index')

In [18]:
# SMILES are the keys
list(node_data.keys())[0]

'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C'

In [19]:
# ChEMBL ID and pKi are the associated values
list(node_data.values())[0]

{'Molecule ChEMBL ID': 'CHEMBL103', 'pKi': 7.515700160653214}

In [20]:
# print(node_data)

## 5. Compute and Compile Edge Data

In [21]:
# We first need to create subset pairs of the SMILES
smis = [] # using ChEMBL provided SMILES
for key,value in node_data.items():
    smis.append(key)

from itertools import combinations
smis_subsets = list(combinations(smis,2))
len(smis_subsets)

81406

In [22]:
# View first 5
smis_subsets[0:5]

[('CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'COc1ccc2c(c1)-c1ccc3c(c1[C@H](c1ccccc1)O2)C(C)=CC(C)(C)N3'),
 ('CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'C#COc1ccc2c(c1OC)-c1ccc3c(c1[C@H](CC=C)O2)C(C)=CC(C)(C)N3'),
 ('CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'CC#C[C@@]1(O)CC[C@@]2(Cc3ccccc3)c3ccc(C(=O)NCc4ccncc4)cc3CC[C@@H]2C1'),
 ('CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'C=CC[C@@H]1Oc2ccc(OC#N)c(OC)c2-c2ccc3c(c21)C(C)=CC(C)(C)N3'),
 ('CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'C#Cc1cccc2c1-c1ccc3c(c1[C@H](CC=C)O2)C(C)=CC(C)(C)N3')]

In [23]:
# create a dictionary, subsets
subsets = {}
for i, (smi1,smi2) in enumerate(smis_subsets):
    field = {}
    field["smi1"] = smi1
    subsets[i] = field
    
    field["smi2"] = smi2
    subsets[i] = field
len(subsets)

81406

In [24]:
# get first key
list(subsets)[0]

0

In [25]:
# get first value
list(subsets.values())[0]

{'smi1': 'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
 'smi2': 'COc1ccc2c(c1)-c1ccc3c(c1[C@H](c1ccccc1)O2)C(C)=CC(C)(C)N3'}

In [26]:
# get smi1
list(subsets.values())[0]['smi1']

'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C'

In [27]:
# add mol objects to our subsets dictionary
for key,value in subsets.items():
    subsets[key].update({"mol1": Chem.MolFromSmiles(value['smi1'])})
    subsets[key].update({"mol2": Chem.MolFromSmiles(value['smi2'])})

In [28]:
list(subsets.keys())[0]

0

In [29]:
list(subsets.values())[0]

{'smi1': 'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
 'smi2': 'COc1ccc2c(c1)-c1ccc3c(c1[C@H](c1ccccc1)O2)C(C)=CC(C)(C)N3',
 'mol1': <rdkit.Chem.rdchem.Mol at 0x7f7ad186ebc0>,
 'mol2': <rdkit.Chem.rdchem.Mol at 0x7f7ad186ee60>}

### Compute Tanimoto Similarity (RDKit fingerprint based)

In [30]:
# compute and add Tanimoto Similarity using default RDKit fingerprints
for key,value in subsets.items():
    fp1 = Chem.RDKFingerprint(value['mol1'])
    fp2 = Chem.RDKFingerprint(value['mol2'])
    tan_sim = round(DataStructs.TanimotoSimilarity(fp1,fp2), 3)
    subsets[key].update({"tan_similarity": tan_sim})

In [31]:
list(subsets.values())[0]

{'smi1': 'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
 'smi2': 'COc1ccc2c(c1)-c1ccc3c(c1[C@H](c1ccccc1)O2)C(C)=CC(C)(C)N3',
 'mol1': <rdkit.Chem.rdchem.Mol at 0x7f6b96c0a6e0>,
 'mol2': <rdkit.Chem.rdchem.Mol at 0x7f6b96c0af80>,
 'tan_similarity': 0.155}

### Compute MCS-based Tanimoto Coefficient (single processor)

In [ ]:
# Add maximum common substructure (MCS)-based Tanimoto Coefficient
# See Vogt, M. et al. J Comput Aided Mol Des 2016, 30 (3), 191–208. 
# https://doi.org/10.1007/s10822-016-9906-3.

############
############

# Compute and add MCS-based similarity
# This may take a very long time...

#############
#############

#def tc_mcs(mol1,mol2):
#    # get maximum common substructure instance
#    mcs = rdFMCS.FindMCS([mol1,mol2],timeout=10) # adding a 10 second timeout for now
    
#    # get number of common bonds
#    mcs_bonds = mcs.numBonds
    
#    # get number of bonds for each
#    # default is only heavy atom bonds
#    mol1_bonds = mol1.GetNumBonds()
#    mol2_bonds = mol2.GetNumBonds()
    
#    # compute MCS-based Tanimoto  
#    tan_mcs = mcs_bonds / (mol1_bonds + mol2_bonds - mcs_bonds)
#    return tan_mcs


# loop through subsets and compute tc_mcs

#for key,value in subsets.items():
#    tan_mcs_value = round(tc_mcs(value['mol1'], value['mol2']), 3)
#    print(key) # to watch progress
#    subsets[key].update({"tan_mcs": tan_mcs_value})

### Compute MCS-based Tanimoto Coefficient (multiprocessing)

In [35]:
# get number of processors
import multiprocessing
print(multiprocessing.cpu_count())

16


In [37]:
# From the Python docs, this below is number of usable CPUs (works on Unix/Linux)
# https://docs.python.org/3/library/multiprocessing.html
# we subtracted 2 from total number, so that we can still easily use computer for other tasks
import os
num_cpus = len(os.sched_getaffinity(0)) - 2
num_cpus

14

In [34]:
# Add maximum common substructure (MCS)-based Tanimoto Coefficient
# See Vogt, M. et al. J Comput Aided Mol Des 2016, 30 (3), 191–208. 
# https://doi.org/10.1007/s10822-016-9906-3.

############
############

# Compute and add MCS-based similarity

# Here are benchmark times with the 10 second timeout in FindMCS and the 81,000 compound gluco pairs:
    # Intel Core i9-9980HK (2.4 GHz x 16) Ubuntu laptop using 14 of the 16 CPUs: ~ 1 hour
    # Intel Core i5-2520M (2.5 GHz x 4) Ubuntu laptop using 3 of the 4 cores: ~ 4 hours

#############
#############

def tc_mcs(mol1,mol2,key):
    # get maximum common substructure instance
    mcs = rdFMCS.FindMCS([mol1,mol2],timeout=10) # adding a 10 second timeout
    
    # get number of common bonds
    mcs_bonds = mcs.numBonds
    
    # get number of bonds for each
    # default is only heavy atom bonds
    mol1_bonds = mol1.GetNumBonds()
    mol2_bonds = mol2.GetNumBonds()
    
    # compute MCS-based Tanimoto
    tan_mcs = mcs_bonds / (mol1_bonds + mol2_bonds - mcs_bonds)
    return key, tan_mcs

# create a list of mol1, mol2, and their dictionary key as tuples
mol_tuples = []
for key, value in subsets.items():
    mol_tuples.append((value['mol1'],value['mol2'], key))

# run multiprocessing on the tc_mcs function
from multiprocessing import Pool

if __name__ == '__main__':
    with Pool(num_cpus) as p: # In our case, num_cpus = 14
        star_map = p.starmap(tc_mcs, mol_tuples)
    for key, tan_mcs in star_map:
        subsets[key].update({"tan_mcs": round(tan_mcs,3)})

In [57]:
list(subsets.values())[0:5]

[{'smi1': 'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'smi2': 'COc1ccc2c(c1)-c1ccc3c(c1[C@H](c1ccccc1)O2)C(C)=CC(C)(C)N3',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x7f6b96c0a6e0>,
  'mol2': <rdkit.Chem.rdchem.Mol at 0x7f6b96c0af80>,
  'tan_similarity': 0.155,
  'tan_mcs': 0.475},
 {'smi1': 'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'smi2': 'C#COc1ccc2c(c1OC)-c1ccc3c(c1[C@H](CC=C)O2)C(C)=CC(C)(C)N3',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x7f6b96c0a9e0>,
  'mol2': <rdkit.Chem.rdchem.Mol at 0x7f6b96c0a8c0>,
  'tan_similarity': 0.16,
  'tan_mcs': 0.45},
 {'smi1': 'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'smi2': 'CC#C[C@@]1(O)CC[C@@]2(Cc3ccccc3)c3ccc(C(=O)NCc4ccncc4)cc3CC[C@@H]2C1',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x7f6b96c0aaa0>,
  'mol2': <rdkit.Chem.rdchem.Mol at 0x7f6b96c0ab00>,
  'tan_similarity': 0.175,
  'tan_mcs': 0.444},
 {'smi1': 'CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C',
  'smi2'

In [58]:
# Keys are integers
list(subsets.keys())[0:5]

[0, 1, 2, 3, 4]

In [59]:
# Here is what the star_map variable looks like
star_map[0:5]

[(0, 0.475),
 (1, 0.45),
 (2, 0.4444444444444444),
 (3, 0.45),
 (4, 0.5277777777777778)]

# 6. Save Data

Save the data, so you don't have to re-compute the tanimoto and MCS similarity again.

In [37]:
# Save the subsets data as a pickle
import pickle
with open('subsets.pickle', 'wb') as outfile:
    pickle.dump(subsets, outfile, pickle.HIGHEST_PROTOCOL)

In [38]:
# Save the node data
with open('node_data.pickle', 'wb') as outfile:
    pickle.dump(node_data, outfile, pickle.HIGHEST_PROTOCOL)